In [4]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load the dataset
file_path = 'Dataset.csv'  # Adjust this path to where your dataset is stored
data = pd.read_csv(file_path)

# Handling missing values for 'Cuisines', as it is crucial for recommendations
data['Cuisines'].fillna('Various', inplace=True)

# Combining relevant features into a single string (feature aggregation)
data['combined_features'] = data.apply(lambda row: f"{row['City']} {row['Cuisines']} {row['Price range']} {row['Rating text']}", axis=1)

# Using TF-IDF Vectorizer to convert text data into a matrix of TF-IDF features
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(data['combined_features'])

# Calculating cosine similarity matrix
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Function to get recommendations based on the cosine similarity score
def get_recommendations(rest_name, cosine_sim=cosine_sim):
    # Get the index of the restaurant that matches the name
    matching_indices = data[data['Restaurant Name'].str.contains(rest_name, case=False, na=False)].index
    if not matching_indices.empty:
        idx = matching_indices[0]
        
        # Get the pairwise similarity scores of all restaurants with that restaurant
        sim_scores = list(enumerate(cosine_sim[idx]))
        
        # Sort the restaurants based on the similarity scores
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        
        # Get the scores of the 10 most similar restaurants
        sim_scores = sim_scores[1:11]
        
        # Get the restaurant indices
        restaurant_indices = [i[0] for i in sim_scores]
        
        # Return the top 10 most similar restaurants
        return data['Restaurant Name'].iloc[restaurant_indices]
    else:
        return "No matching restaurants found. Please check the name and try again."

# Example usage
recommendations = get_recommendations('Thai Kitchen')  # Replace 'Some Restaurant Name' with a valid name from your dataset
print(recommendations)


/var/folders/q5/3gdpzhz56vdb5ymrs8jgrsdc0000gn/T/ipykernel_88530/4036880774.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Cuisines'].fillna('Various', inplace=True)


137                           The Taj of India
130                         Frog Hollow Tavern
133                       Nacho Mamas Burritos
142               El Kiosco Mexican Restaurant
136                             The Chop House
127                            Farmhaus Burger
131    Giuseppe's Pizza & Italian Specialities
129                         Sconyers Bar B Que
140                     Rhinehart's Oyster Bar
139                                    Pho Bac
Name: Restaurant Name, dtype: object
